<a href="https://colab.research.google.com/github/Nithish4603/Text-Summarization/blob/main/Text_summarization_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:
https://github.com/Shivanandroy/simpleT5 ,
https://github.com/karanthakkar97/Text-summarization-with-Seq2Seq/blob/main/Text-summarizer.ipynb,
https://snrspeaks.medium.com/abstractive-summarization-with-simplet5-%EF%B8%8F-344a78f73265,
https://www.youtube.com/watch?v=jgKj-7v2UYU


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nltk

print('The nltk version is {}.'.format(nltk.__version__)) 

The nltk version is 3.7.


In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
!pip install -U nltk 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importing the Dataset

In [ ]:
summary = pd.read_csv('/content/drive/MyDrive/training/news_summary.csv', encoding='iso-8859-1')
raw = pd.read_csv('/content/drive/MyDrive/training/news_summary_more.csv', encoding='iso-8859-1')

In [ ]:
pre_data1 =  raw.iloc[:,0:2].copy()
pre_data2 = summary.iloc[:,0:6].copy()
pre_data2['text'] = pre_data2['author'].str.cat(pre_data2['date'].str.cat(pre_data2['read_more'].str.cat(pre_data2['text'].str.cat(pre_data2['ctext'], sep = " "), sep =" "),sep= " "), sep = " ")

In [ ]:
#taking only the required columns
pre_data = pd.DataFrame()
pre_data['text'] = pd.concat([pre_data1['text'], pre_data2['text']], ignore_index=True)
pre_data['summary'] = pd.concat([pre_data1['headlines'],pre_data2['headlines']],ignore_index = True)

In [ ]:
pre_data.head(2)

,text,summary
0,"Saurav Kant, an alumnus of upGrad and IIIT-B's...",upGrad learner switches to career in ML & Al w...
1,Kunal Shah's credit card bill payment platform...,Delhi techie wins free food from Swiggy for on...


In [ ]:
pre_data = pre_data.dropna()
pre_data.head()

,text,summary
0,"Saurav Kant, an alumnus of upGrad and IIIT-B's...",upGrad learner switches to career in ML & Al w...
1,Kunal Shah's credit card bill payment platform...,Delhi techie wins free food from Swiggy for on...
2,New Zealand defeated India by 8 wickets in the...,New Zealand end Rohit Sharma-led India's 12-ma...
3,"With Aegon Life iTerm Insurance plan, customer...",Aegon life iTerm insurance plan helps customer...
4,Speaking about the sexual harassment allegatio...,"Have known Hirani for yrs, what if MeToo claim..."


In [ ]:
#renaming the column names
pre_data = pre_data.rename(columns={'text': 'source_text', 'summary': 'target_text'})
pre_data

,source_text,target_text
0,"Saurav Kant, an alumnus of upGrad and IIIT-B's...",upGrad learner switches to career in ML & Al w...
1,Kunal Shah's credit card bill payment platform...,Delhi techie wins free food from Swiggy for on...
2,New Zealand defeated India by 8 wickets in the...,New Zealand end Rohit Sharma-led India's 12-ma...
3,"With Aegon Life iTerm Insurance plan, customer...",Aegon life iTerm insurance plan helps customer...
4,Speaking about the sexual harassment allegatio...,"Have known Hirani for yrs, what if MeToo claim..."
...,...,...
102910,"Mansha Mahajan 24 Feb 2017,Friday http://india...",Rasna seeking ?250 cr revenue from snack categ...
102911,"Dishant Sharma 03 Aug 2017,Thursday http://ind...",Sachin attends Rajya Sabha after questions on ...
102912,"Tanya Dhingra 03 Aug 2017,Thursday http://www....",Shouldn't rob their childhood: Aamir on kids r...
102913,"Pragya Swastik 07 Dec 2016,Wednesday http://in...","Asha Bhosle gets ?53,000 power bill for unused..."


In [ ]:
#adding prefix "summarize" to the source_text column
pre_data['source_text'] = "summarize: " + pre_data['source_text']

In [ ]:
pre_data

,source_text,target_text
0,"summarize: Saurav Kant, an alumnus of upGrad a...",upGrad learner switches to career in ML & Al w...
1,summarize: Kunal Shah's credit card bill payme...,Delhi techie wins free food from Swiggy for on...
2,summarize: New Zealand defeated India by 8 wic...,New Zealand end Rohit Sharma-led India's 12-ma...
3,summarize: With Aegon Life iTerm Insurance pla...,Aegon life iTerm insurance plan helps customer...
4,summarize: Speaking about the sexual harassmen...,"Have known Hirani for yrs, what if MeToo claim..."
...,...,...
102910,"summarize: Mansha Mahajan 24 Feb 2017,Friday h...",Rasna seeking ?250 cr revenue from snack categ...
102911,"summarize: Dishant Sharma 03 Aug 2017,Thursday...",Sachin attends Rajya Sabha after questions on ...
102912,"summarize: Tanya Dhingra 03 Aug 2017,Thursday ...",Shouldn't rob their childhood: Aamir on kids r...
102913,"summarize: Pragya Swastik 07 Dec 2016,Wednesda...","Asha Bhosle gets ?53,000 power bill for unused..."


In [ ]:
len(pre_data)

102797

In [ ]:
#As the dataset is very large we are considering only first 25000 rows for our training data
main_data = pre_data[:25000]
#we are considering 1000 rows for test data
test_data = pre_data[25001:26001]

In [ ]:
len(main_data)

25000

In [ ]:
len(test_data)

1000

In [ ]:
pip install --upgrade simplet5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 30.6 MB/s 
     |████████████████████████████████| 3.5 MB 34.2 MB/s 
     |████████████████████████████████| 527 kB 71.9 MB/s 
     |████████████████████████████████| 512 kB 68.5 MB/s 
     |████████████████████████████████| 829 kB 76.6 MB/s 
     |████████████████████████████████| 952 kB 69.4 MB/s 
     |████████████████████████████████| 7.6 MB 69.2 MB/s 
     |████████████████████████████████| 182 kB 81.9 MB/s 
     |████████████████████████████████| 880 kB 69.4 MB/s 
  Created wheel for simplet5: filename=simplet5-0.1.4-py3-none-any.whl size=6873 sha256=754a97cacb17445d55933894283d693b651d478ae4094359c63d5fe77773a3dc
  Stored in directory: /root/.cache/pip/wheels/32/a6/37/e007cfdbceac6347aa13bc2a881ac87d2a3d6e4202663ee311
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491071 sha256=80979d6b12f67914cbf8d8a45abb33b8

In [ ]:
#splitting the data
from sklearn.model_selection import train_test_split
train_df, eval_df = train_test_split(main_data, test_size=0.1)
train_df.shape, eval_df.shape

((22500, 2), (2500, 2))

In [ ]:
# import
from simplet5 import SimpleT5

# instantiate
model = SimpleT5()

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
# load (supports t5, mt5, byT5 models)
model.from_pretrained("t5","t5-base")

# train
model.train(train_df=train_df, # pandas dataframe with 2 columns: source_text & target_text
            eval_df=eval_df, # pandas dataframe with 2 columns: source_text & target_text
            source_max_token_len = 256, 
            target_max_token_len = 128,
            batch_size = 8,
            max_epochs = 3,
            use_gpu = True,
            outputdir = "outputs",
            early_stopping_patience_epochs = 0,
            precision = 32
            )

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


...

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
model.load_model("t5","/content/outputs/simplet5-epoch-2-train-loss-0.888-val-loss-1.0644", use_gpu=False)


In [ ]:
test_data['source_text'][25001]

"summarize: India's largest private lender by market capitalisation, HDFC Bank, on Saturday posted an 18.2% increase in its profit at Ã¢Â\x82Â¹4,601 crore for the quarter ended June. This is the bank's slowest profit growth in five quarters and missed analysts' estimates. The bank's net interest income or the core income from operations, rose 15.4% to Ã¢Â\x82Â¹10,813.6 crore."

In [ ]:
# predict
model.predict(test_data['source_text'][25001])

["HDFC Bank's profit rises 18.2% to 14,601 cr in June quarter"]

In [ ]:
test_data['target_text'][25001]

"HDFC Bank's June quarter profit grows 18% to Ã¢Â\x82Â¹4,601 crore"

In [ ]:
#printing the first 10 predictions on test dataset
for i in range(25001,25010):
    print("news:",test_data['source_text'][i])
    print("original summary:",test_data['target_text'][i])
    print("predicted summary:",model.predict(test_data['source_text'][i]))
    print("\n")

news: summarize: India's largest private lender by market capitalisation, HDFC Bank, on Saturday posted an 18.2% increase in its profit at Ã¢ÂÂ¹4,601 crore for the quarter ended June. This is the bank's slowest profit growth in five quarters and missed analysts' estimates. The bank's net interest income or the core income from operations, rose 15.4% to Ã¢ÂÂ¹10,813.6 crore.
original summary: HDFC Bank's June quarter profit grows 18% to Ã¢ÂÂ¹4,601 crore
predicted summary: ["HDFC Bank's profit rises 18.2% to 14,601 cr in June quarter"]


news: summarize: The government on Friday indicated that 25% of the ATMs run by state-owned banks may be vulnerable to fraud as they are running on outdated software. However, it did not disclose details of such ATMs run by private sector lenders. The government also informed the Parliament that the total number of functional ATMs constitutes nearly 89% of the total ATMs.
original summary: 25% ATMs of state-run banks running on outdated software
predic

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
#calculating BleuScore for the Test dataset
#Test dataset range is 25001 to 26001 so N is 1000
from nltk.translate.bleu_score import sentence_bleu
def calculate_scores(N=1000):    
    bscore=0
    for i in range(25001,26001):
        ref=test_data['target_text'][i]
        hypo=str(model.predict(test_data['source_text'][i]))
        bscore+=sentence_bleu([ref],hypo,weights=(1,0,0,0))
    print("BLEU:%.4f "%(bscore/1000))

In [ ]:
calculate_scores()

/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


BLEU:0.7419 
